# Location Designator
This example will show you waht location designator are, how to use them and what they are capable of. 

Location Deisgnator are used to semantically describe locations in the world. You could, for example, create a location designator that describes every position where a robot can be placed without colliding with the environment. Location designator can describe locations for:

   * Visibility 
   * Reachability
   * Occupany 
   * URDF Links (for example a table)

To find locations that fit the given constrains, location designator create Costmaps. Costmaps are a 2D distribution that have a value greater than 0 for every position that fits the costmap criteria.

Location designator work similar to other designator, meaning you have to create a location designator description which describes the location. This description can then be resolved to the actual 6D pose on runtime.

## Occupancy

We will start with a simple location designator that describes a location where the robot can be placed without colliding with the environment. To do this we need a BulletWorld since the costmaps are mostly created from the current state of the BulletWorld. 

In [1]:
from pycram.bullet_world import BulletWorld, Object

world = BulletWorld()
kitchen = Object("kitchen", "environment", "kitchen.urdf")

Unknown tag "material" in /robot[@name='plane']/link[@name='planeLink']/collision[1]
Unknown tag "contact" in /robot[@name='plane']/link[@name='planeLink']
Scalar element defined multiple times: limit
Unknown tag "material" in /robot[@name='plane']/link[@name='planeLink']/collision[1]
Unknown tag "contact" in /robot[@name='plane']/link[@name='planeLink']
Scalar element defined multiple times: limit


In [12]:
world.exit()

Next up we will create the location designator description, the ```CostmapLocation``` that we will be using needs a target as parameter. This target describes what the location designator is for, this could either be a pose or object that the robot should be able to see or reach.

In this case we only want poses where the robot can be placed, this is the default behaviour of the location designator which we will be extending later. 

In [2]:
from pycram.designators.location_designator import CostmapLocation

target = kitchen.get_position_and_orientation()

location_description = CostmapLocation(target)

pose = location_description.resolve()

print(pose)

CostmapLocation.Location(pose=[[-1.159999966621399, -0.25999999046325684, -0.0], [0.0, 0.0, 0.11002368521143333, 0.9939289656170079]], reachable_arms=None)


## Reachable
Next we want to locations from where the robot can reach a specific point, like an object the robot should pick up. This can also be done with the ```CostmapLocation``` description, but this time we need to provide an additional argument. The additional argument is the robo which should be able to reach the pose. 

Since a robot is needed we will use the PR2 and use a milk as a target point for the robot to reach. The torso of the PR2 will be set to 0.2 since otherwise the arms of the robot will be too low to reach on the countertop.


In [3]:
pr2 = Object("pr2", "robot", "pr2.urdf")
pr2.set_joint_state("torso_lift_joint", 0.2)
milk = Object("milk", "milk", "milk.stl", position=[1.3, 1, 0.9])


In [4]:
from pycram.designators.location_designator import CostmapLocation
from pycram.designators.object_designator import BelieveObject

target = BelieveObject(names=["milk"]).resolve()
robot_desig = BelieveObject(names=["pr2"]).resolve()

location_description = CostmapLocation(target=target, reachable_for=robot_desig)

print(location_description.resolve())

CostmapLocation.Location(pose=[[0.7199999690055847, 0.9599999785423279, 0.0], [0.0, 0.0, 0.03442146029735499, 0.9994074059516458]], reachable_arms=['left', 'right'])


As you can see we get a pose near the countertop where the robot can be placed without colliding with it. Furthermore, we get a list of arms with which the robot can reach the given object.

## Visibile
The ```CostmapLocation``` can also find position from which the robot can see a given object or location. This is very similar to how rechable locations are described, meaning we provide a object designator or a pose and a robot designator but this time we use the ```visible_for``` parameter.  

For this example we need the milk as well as the PR2, so if you did not spawn them during the previous location designator you can spawn them with the following cell.  

In [ ]:
pr2 = Object("pr2", "robot", "pr2.urdf")
milk = Object("milk", "milk", "milk.stl", position=[1.3, 1, 0.9])

In [5]:
from pycram.designators.location_designator import CostmapLocation
from pycram.designators.object_designator import BelieveObject

target = BelieveObject(names=["milk"]).resolve()
robot_desig = BelieveObject(names=["pr2"]).resolve()

location_description = CostmapLocation(target=target, visible_for=robot_desig)

print(location_description.resolve())

CostmapLocation.Location(pose=[[0.6999999284744263, 0.7799999713897705, 0.0], [0.0, 0.0, 0.17481945732557833, 0.9846005064697003]], reachable_arms=None)


## Semantic 
Semantic location designator are used to create location descriptions for semantic entities, like a table. An example of this is: You have a robot that picked up an object and should place it on a table. Semantic location designator then allows to find poses that are on this table.

Semantic location designator need an object from which the target entity is a part and the urdf link representing the entity. In this case we want a position on the kitchen island, so we have to provide the kitchen object designator since the island is a part of the kitchen and the link name of the island surface. 

For this example we need the kitchen as well as the milk. If you spawned them in one of the previous examples you don't need to execute the following cell.

In [8]:
kitchen = Object("kitchen", "environment", "kitchen.urdf")
milk = Object("milk", "milk", "milk.stl")

In [6]:
from pycram.designators.location_designator import SemanticCostmapLocation 
from pycram.designators.object_designator import BelieveObject

kitchen_desig = BelieveObject(names=["kitchen"]).resolve()
milk_desig = BelieveObject(names=["milk"]).resolve()

location_description = SemanticCostmapLocation(urdf_link_name="kitchen_island_surface", 
                                               part_of=kitchen_desig,
                                              for_object=milk_desig)

print(location_description.resolve())

SemanticCostmapLocation.Location(pose=[[-1.2074999809265137, 1.019200086593628, 0.9398907270729542], [0.0, 0.0, 0.6339889056055381, 0.7733421413379024]])


## Location Designator as Generator
Location designator descriptions implement an iter method, so they can be used as generators which generate valid poses for the location described in the description. This can be useful if the first pose does not work for some reason. 

We will see this at the example of a location designator for visibility. For this example we need the milk, if you already have a milk spawned in you world you can ignore the following cell.

In [ ]:
milk = Object("milk", "milk", "milk.stl")

In [9]:
from pycram.designators.location_designator import CostmapLocation
from pycram.designators.object_designator import BelieveObject

target = BelieveObject(names=["milk"]).resolve()
robot_desig = BelieveObject(names=["pr2"]).resolve()

location_description = CostmapLocation(target=target, visible_for=robot_desig)

for pose in location_description:
    print(pose.pose)

[[0.6999999284744263, 0.7799999713897705, 0.0], [0.0, 0.0, 0.17481945732557833, 0.9846005064697003]]
[[0.2999999523162842, 1.3799999952316284, 0.0], [-0.0, 0.0, 0.1805774242634438, -0.9835607728281869]]
[[0.2999999523162842, 1.440000057220459, 0.0], [-0.0, 0.0, 0.2057729085728114, -0.9785997701294873]]
[[0.2999999523162842, 1.0800000429153442, 0.0], [-0.0, 0.0, 0.0399044143055408, -0.9992035016546588]]
[[0.2999999523162842, 1.3600000143051147, 0.0], [-0.0, 0.0, 0.17191943937986326, -0.9851110122028448]]
[[0.2999999523162842, 1.340000033378601, 0.0], [-0.0, 0.0, 0.1631368446021929, -0.9866034512068361]]
[[0.2999999523162842, 1.3199999332427979, 0.0], [-0.0, 0.0, 0.154233313726437, -0.9880344553389636]]
[[0.2999999523162842, 1.2999999523162842, 0.0], [-0.0, 0.0, 0.1452131165515644, -0.9894003996266536]]
[[0.2999999523162842, 1.2799999713897705, 0.0], [-0.0, 0.0, 0.13608080282361212, -0.9906977415452612]]
[[0.2999999523162842, 1.2599999904632568, 0.0], [-0.0, 0.0, 0.12684141287795903, -0.

## Accessing Locations
Accessing describes a location from which the robot can open a drawer. The drawer is specified by a ObjetcPart designator which describes the handle of the drawer.

At the moment this location designator only works in the apartment environment, so please remove the kitchen if you spawned it in a previous example. Furthermore, we need a robot so we also spawn the pr2 if it isn't spawned already.

In [2]:
kitchen.remove()

In [3]:
apartment = Object("apartment", "environment", "apartment.urdf")

Unknown tag "material" in /robot[@name='apartment']/link[@name='coffe_machine']/collision[1]
Unknown tag "material" in /robot[@name='apartment']/link[@name='coffe_machine']/collision[1]


In [10]:
pr2 = Object("pr2", "robot", "pr2.urdf")
pr2.set_joint_state("torso_lift_joint", 0.25)

Unknown attribute "type" in /robot[@name='pr2']/link[@name='base_laser_link']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='wide_stereo_optical_frame']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='narrow_stereo_optical_frame']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='laser_tilt_link']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='base_laser_link']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='wide_stereo_optical_frame']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='narrow_stereo_optical_frame']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='laser_tilt_link']


In [11]:
from pycram.designators.object_designator import *
from pycram.designators.location_designator import *

apartment_desig = BelieveObject(names=["apartment"])
handle_desig = ObjectPart(names=["handle_cab10_t"], part_of=apartment_desig.resolve())
robot_desig = BelieveObject(names=["pr2"])

access_location = AccessingLocation(handle_desig.resolve(), robot_desig.resolve()).resolve()
print(access_location.pose)

([1.8074915409088135, 2.7473597526550293, 0.0], [-0.0, 0.0, 0.5893608435105937, -0.8078699128798408])
